In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, TimestampType, DoubleType
from pyspark.sql.types import StructField, StructType, StringType

spark = (
    
            SparkSession
            .builder
            .appName('SparkCourse') 
            .master('local[*]') 
            .config('spark.dynamicAllocation.enabled', 'false')
            .config('spark.sql.adaptive.enabled', 'false')
            .getOrCreate()
    )

sc = spark.sparkContext

spark

In [1]:
9500 + 3300 + 310 + 350 + 140 

13600

In [2]:
1400 /20

70.0

In [2]:
data_path = '../data/'

In [6]:
taxi_schema = StructType([
    StructField('VendorID', IntegerType(), True),
    StructField('tpep_pickup_datetime', TimestampType(), True),
    StructField('tpep_dropoff_datetime', TimestampType(), True),
    StructField('passenger_count', DoubleType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('RatecodeID', DoubleType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('PULocationID', IntegerType(), True),
    StructField('DOLocationID', IntegerType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('congestion_surcharge', DoubleType(), True),
    StructField('airport_fee', DoubleType(), True),
])

In [8]:
yellow_taxi_df = (
    spark
        .read
        .option('header', 'true')
        .schema(taxi_schema)
        .csv(data_path + 'YellowTaxis_202210.csv')
)

In [15]:
green_taxi_df = (spark
                    .read
                    .option('header', 'true')
                    .option('delimiter', '\t')
                    .csv(data_path + 'GreenTaxis_202210.csv')
    )

green_taxi_df.createOrReplaceTempView('GreenTaxis')

In [10]:
yellow_taxi_df.createOrReplaceTempView('YellowTaxis')

In [11]:
output_df = spark.sql(
    'SELECT * FROM YellowTaxis WHERE PULocationID=171'
)

In [12]:
output_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-10-01 11:17:23|  2022-10-01 12:08:50|            1.0|          9.4|      99.0|                 N|         171|         263|           1|       35.2|  0.0|    0.5|       0.

In [ ]:
spark.sql(
    
)

In [16]:
30/46

0.6521739130434783

In [17]:
0.7 * 46

32.199999999999996

23/08/16 10:20:00 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1635545 ms exceeds timeout 120000 ms
23/08/16 10:20:01 WARN SparkContext: Killing executors is not supported by current scheduler.
23/08/16 10:20:05 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.